### 第5章 

ライブラリのimport

In [21]:
from openai import OpenAI
import yaml
import json

【注意】下記実行前にREADME.mdに従いルートフォルダにconfig.yamlを作成してください。

In [22]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

クライアントの作成

In [23]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

サンプルシナリオ例

In [ ]:
instruction = """

# Role
あなたはコンサルティングファームの週次ニュース調査・事例突合・レポート配信を行うツール駆動エージェントです。
<operating_mode>
- すべての作業はツール呼び出しで完結する。
- 最終出力は必ず send_email の tool call で終了する。
</operating_mode>

# Task
<workflow>
Phase1: 入力理解・期間確定
Phase2: web_search でニュース収集(複数クエリ)
Phase3: 重要度選別・要約・タグ付け
Phase4: case_search で社内事例突合
Phase5: report_request_approval でレポート生成・承認依頼
Phase6: report_request_approval → approved 確認(承認されなければ指摘を反映し再依頼)
Phase7: send_email で配信(最終)
</workflow>

# Input
<input_schema>
(【略】InputのJSONスキーマとそれぞれのプロパティ説明)
</input_schema>

# Tool
Taskのworkflow内におけるPhaseを思考中に必ず確認し、適切なツールを呼び出して作業を進める。

<tool_usage_rules>
- 根拠になった情報は検索結果および格納先のURLを[<リンク>]として必ず付与する。
</tool_usage_rules>


## report_request_approval
出力にはreport_template_markdownテンプレートを用いる
<report_template_markdown>
## サマリ(3〜6行)
- (最重要トピックを最大3点)

## 今週の重要動向(最大5件)
- [重要度: 高/中/低] 見出し

## 過去事例との突合(最大5件)
- 事例ID/パス: ...

## リスク/機会と推奨アクション(最大5件)
- (アクションは「誰が/何を/いつまでに」が分かる書き方)

## 前提・不確実性(必須)
- (推測は推測と明記。未確認は未確認と明記)
</report_template_markdown>

# Policy
<quality_and_grounding>
- 根拠のない断定は禁止。日付・数字・固有名詞は特に厳密。
- 矛盾があれば追加検索し、解消できなければ不一致として記載。
</quality_and_grounding>

<privacy_and_handling>
- 社内事例DBの詳細を社外共有しない前提で、メール本文は要約＋参照ID/パスに留める。
</privacy_and_handling>
"""

report_request_approval_description = """
Purpose: 収集済みニュースと社内事例をもとにレポートMarkdownを生成し、承認依頼として保存・申請します。

Use when:
- Phase5、および承認が下りなかった場合の再承認依頼時
- 対象顧客・期間・想定読者などの前提と必要情報（ニュース要約・事例・推奨アクション）が揃っている

Do not use when:
- Phase5以外
- 収集・分析が未完了で、まず要件や前提条件の確認が必要な状態

Notes:
- 失敗時: 生成したレポートは保持したまま簡潔に指摘内容を反映し、再試行。
"""

report_markdown_description = """
以下の点を遵守してレポートを作成してください。
- レポートはreport_template_markdownテンプレートに従い、冗長な叙述を避ける(箇条書き中心)。
- 入力スコープ(顧客・業界・地域・論点・期間)を逸脱しない。
- 不足情報は「仮定」として明示。
"""


In [5]:
tools = [
    {"type": "web_search"},
    {
        "type": "function",
        "name": "report_request_approval",
        "description": report_request_approval_description,
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
            "report_markdown": {
                "type": "string",
                "description": report_markdown_description
            }
            },
            "required": ["report_markdown"],
            "additionalProperties": False
        }
    }
]

In [6]:
messages=[
        {"role": "developer", "content": instruction},
        {"role": "user", "content": "鉄道業界の架空のレポートが出せる状態になった想定で、Phase5に入ったとして、ツール選択してください。レポートは架空のものをそれっぽく作ってください。"}
    ]

response = client.responses.create(
    instructions=instruction,
    model="gpt-5-mini",
    input=messages,
    tools=tools,
    tool_choice="required"
)

response_type = response.output[1].type
function_name = response.output[1].name
arguments = json.loads(response.output[1].arguments)
id = response.output[1].call_id

print("response_type: ", response_type)
print("function_name: ", function_name)
print("argument: ", arguments)
print("id: ", id)

response_type:  function_call
function_name:  report_request_approval
argument:  {'report_markdown': '## サマリ(3〜6行)\n- 自動列車運転（ATO）・自動化の実証実験が複数都市でフェーズ移行しており、短中期で運行コスト低減とサービス拡張が見込まれる（架空）。[ニュースソース: 社内スクレイピング/2025-12-30]\n- ポストパンデミックの乗客回復は進むが、通勤中心の需要構造変化により運賃収入の構成比が変化、ダイナミックプライシング導入の検討が想定される（架空）。[報告格納先: /internal/rail/market/2025-Q4]\n- 気候変動に伴う極端気象リスクがインフラ影響を拡大させており、耐候性投資・非常時対応力強化の必要性が高まっている（架空）。[参照: 社内気候レポート/2025-11]\n\n## 今週の重要動向(最大5件)\n- [重要度: 高] 都市鉄道でのATOフェーズ2試験開始 — 時間帯別自動運転拡大と運行最適化のPoCが開始（架空）。[リンク: /news/ato-poc-2025-12-28]\n- [重要度: 中] 乗客数回復の局所差 — 都市中心部は回復基調だが郊外通勤は未だ低水準で、運賃施策の見直しが必要（架空）。[リンク: /news/ridership-2025-12-25]\n- [重要度: 中] 輸送需要のモーダルシフト（貨物） — 環境規制とトラック運賃上昇で一部貨物が鉄道へ移行。受け入れインフラ整備がボトルネック（架空）。[リンク: /news/freight-shift-2025-12-20]\n- [重要度: 中] 車両・半導体供給遅延の継続 — 新型車両導入と設備更新計画に影響（架空）。[リンク: /news/supplychain-2025-12-22]\n- [重要度: 低] 水素・蓄電池車両の実証進展 — 定期運行に向けた技術検証が進むがコスト課題は残存（架空）。[リンク: /news/h2-battery-2025-12-15]\n\n## 過去事例との突合(最大5件)\n- 事例ID/パス: CASE-RAIL-2022-01: ATO導入支援（某都市鉄道） — PoC設計／安全評価

## Reasoning過程の取り出し

In [ ]:
# 会話履歴の格納先を生成
conversation = client.conversations.create()

# ツール使用結果とReasoning過程を会話履歴に含める設定でLLMにテキスト生成リクエスト
response = client.responses.create(
  model="gpt-5-nano",
  input=[
    {"role": "system", "content": "要求された内容だけ答えてください。"},
    {"role": "user", "content": "LLMのRAGに関連するキーワードとその簡易解説を1つ出力して。"}
  ],
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  tool_choice="required",
  include=["web_search_call.action.sources",  "reasoning.encrypted_content"],
  conversation=conversation.id
)

レスポンスの概要

In [79]:
for resp in response.output:
    # print(resp)
    if resp.type == 'reasoning':
        print(json.dumps({"type": resp.type, "summary": [s.text for s in resp.summary]}, ensure_ascii=False, indent=2))
    elif resp.type == 'web_search_call':
        print(json.dumps({"type": resp.type, "query": resp.action.query}, ensure_ascii=False, indent=2))
    elif resp.type == 'message':
        print(json.dumps({"type": resp.type, "role": resp.role, "text": [c.text for c in resp.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "reasoning",
  "summary": [
    "**Researching vector database in Japanese**\n\nI’m focusing on a keyword related to LLM's RAG. The term \"ベクトルデータベース\" (vector database) feels appropriate, as it pertains to storing and retrieving high-dimensional vectors used in searches. I plan to provide a simple explanation about how it stores embedding vectors for documents and retrieves related documents using query embeddings with cosine similarity. Since I need to use the web.run tool to find reliable sources, I’ll also consider the citation requirements for supporting statements.",
    "**Searching for RAG vector database purpose**\n\nI’m planning to perform a search for \"RAG vector database purpose\" or \"vector database for RAG.\" I want to find reliable sources like academic papers or blogs, including the \"RAG: Retrieval-Augmented Generation\" paper by Facebook AI, or details about the \"Faiss\" vector search library. I’ll also aim to site concepts about vector databases storin

conversationへの格納結果

In [80]:
items = client.conversations.items.list(conversation.id, limit=10)

for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "role": item.role, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "message",
  "role": "assistant",
  "text": [
    "- キーワード: ベクトルデータベース\n- 簡易解説: 高次元の埋め込みベクトルを格納・検索するデータベース。RAGでは文書を埋め込み化してベクトルデータベースに格納し、クエリの埋め込みと近似最近傍検索で最も関連性の高い文書を取り出して、LLMの生成にコンテキストとして提供する。 ([en.wikipedia.org](https://en.wikipedia.org/wiki/Vector_database?utm_source=openai))"
  ]
}
{
  "type": "reasoning",
  "summary": []
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation vector database embedding"
}
{
  "type": "reasoning",
  "summary": [
    "**Researching vector database in Japanese**\n\nI’m focusing on a keyword related to LLM's RAG. The term \"ベクトルデータベース\" (vector database) feels appropriate, as it pertains to storing and retrieving high-dimensional vectors used in searches. I plan to provide a simple explanation about how it stores embedding vectors for documents and retrieves related documents using query embeddings with cosine similarity. Since I need to use the web.run tool to find reliable sources, I’ll also consider the c

reasoning itemだけを削除

In [81]:
reasoning_msg_ids = [itm.id for itm in items.data if itm.type == "reasoning"]

for msg_id in reasoning_msg_ids:
   new_conversation = client.conversations.items.delete(conversation_id=conversation.id, item_id=msg_id)

items = client.conversations.items.list(new_conversation.id, limit=10)
 
for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "message",
  "text": [
    "- キーワード: ベクトルデータベース\n- 簡易解説: 高次元の埋め込みベクトルを格納・検索するデータベース。RAGでは文書を埋め込み化してベクトルデータベースに格納し、クエリの埋め込みと近似最近傍検索で最も関連性の高い文書を取り出して、LLMの生成にコンテキストとして提供する。 ([en.wikipedia.org](https://en.wikipedia.org/wiki/Vector_database?utm_source=openai))"
  ]
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation vector database embedding"
}
{
  "type": "message",
  "text": [
    "LLMのRAGに関連するキーワードとその簡易解説を1つ出力して。"
  ]
}
{
  "type": "message",
  "text": [
    "要求された内容だけ答えてください。"
  ]
}


会話履歴の圧縮

###### ※この例ではサンプルが小さいため、圧縮しても効果が薄いor元よりもトークンが大きくなる点に注意

In [113]:
# ダミーの会話を用意
messages = [
    {"role":"system","content":"あなたはLLMの専門家です。具体情報を交えて短め(最大50字程度)で答えてください。"},
    {"role":"user","content":"私は中学生です。GPT-5.2とGPT-4.1の位置づけを、用途別に教えてください。"},
    {"role":"assistant","content":"GPT-5.2は推論対応の旗艦で、複雑なコーディングやエージェントに強いです。"},
    {"role":"user","content":"低コスト重視で遅延も抑えたいとき、GPT-5系はどう選びますか。"},
    {"role":"assistant","content":"GPT-5 miniは速く安価で定型タスク向きで、さらに軽量ならGPT-5 nanoが候補です。"},
    {"role":"user","content":"長文を大量に扱う場合、推論なしで高文脈の選択肢はありますか。"},
    {"role":"assistant","content":"推論ステップ不要ならGPT-4.1が約1M文脈で有力で、難問はGPT-5系も併用します。"},
    {"role":"user","content":"最新のモデルはGoogleだと何だろう？調べてくれる？"}
]

response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    input=messages,
    store=True
)

print("結果:", response.output_text)

結果: 最新は Gemini 3。Pro/Deep Think、Ultraはプラン別。 ([blog.google](https://blog.google/products/gemini/gemini-3/?utm_source=openai))


In [114]:
# まずは非圧縮版で実行
normal_response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    previous_response_id=response.id,
    input=[{"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"}],
)
print("非圧縮の結果:", normal_response.output_text)
print("非圧縮のインプットトークン:", normal_response.usage.input_tokens)


非圧縮の結果: 推論対応の旗艦で、複雑なコーディングやエージェントに強いです。
非圧縮のインプットトークン: 4962


In [115]:
# これまでの user message + output を compact する
compacted = client.responses.compact(
    model="gpt-5-mini",
    previous_response_id=resp.id,
)
compacted_messages = compacted.output

# 次のターンの入力を追加
compacted_messages.append({"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"})

# 次に圧縮版で実行
compacted_response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    input=compacted_messages,
)

print("圧縮後の結果:", compacted_response.output_text)
print("圧縮後のインプットトークン:", compacted_response.usage.input_tokens)

c:\Users\hirosatogamo\.virtualenvs\pipenv-slS7I_su\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `literal['output_text']` - serialized value may not be as expected [field_name='type', input_value='input_text', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `ResponseOutputRefusal` - serialized value may not be as expected [field_name='content', input_value=ResponseOutputText(annota...ut_text', logprobs=None), input_type=ResponseOutputText])
  return self.__pydantic_serializer__.to_python(


圧縮後の結果: 最新情報に基づく要点だけをお伝えします。GPT-5.2は公式発表によると、以下の領域で従来モデルより強化されています（2025年12月頃のリリース、一般利用はChatGPTとAPIで順次提供開始）。

- 汎用的な生産性・タスク処理の向上
  - スプレッドシート作成、プレゼンテーション作成、コード作成、画像の理解、長い文脈の理解、ツールの活用、複雑で多段階のプロジェクトの処理能力が向上。これらは日常的な業務ワークフロー全体での効率化を狙っています。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))

- 長文・長文ドキュメントの処理能力（長文推論と統合）
  - 長文コンテキスト推論で新しい水準を達成し、実務データの深い分析・統合が可能になるとしています。 MRCR v2 の評価で256kトークン相当の長文でも高い精度を示すとされています。実務での深い分析・多ソースの統合に適しています。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))

- ツール呼び出しと複数アクションの統合（エージェント的運用）
  - 複数のツールを横断して使うワークフローの安定性と効率が向上。 Tau2-bench Telecomでのツール活用が高精度（約98.7%）に達するなど、長い対話・複数ステップのタスクでの信頼性が改善されています。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))

- コード生成・エージェント的能力の強化（GPT-5.2-Codex など）
  - エージェント的なコード作成や大規模なコード変更・リファクタリング、Windows環境でのパフォーマンス改善など、専門的な開発領域での能力が向上。コード関連のタスクで特に強化されています。 ([openai.com](https://openai.com/index/introducing-gpt-5-2-codex/?utm_source=openai))

- 安全性